In [4]:
%cd drive/MyDrive/protein-bert/proteinbert/


!pip install git+'https://github.com/miladansari/protein_bert.git'
!git clone https://github.com/nadavbra/shared_utils.git

#import related libraries

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import datetime
from pandas import read_csv
%load_ext tensorboard


#reading files.

test = read_csv('hemolytic.test.csv',skipinitialspace=True)
train = read_csv('hemolytic.train.csv',skipinitialspace=True)

seqs=train['seq']
seqs_test=test['seq']

#set sequence length to 37 (the longest sequence in dataset is 35 and the model will add <start> and <end> token to the sequence.

seq_len=37
batch_size=32

from proteinbert import load_pretrained_model
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

#load pretrained model.
pretrained_model_generator, input_encoder = load_pretrained_model(local_model_dump_dir='/content/drive/MyDrive/protein-bert/proteinbert')
model = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(seq_len))

#extract local representaion (Embeddings).
X = input_encoder.encode_X(seqs, seq_len)
local_representations, global_representations= model.predict(X, batch_size = batch_size)

X_test= input_encoder.encode_X(seqs_test, seq_len)
local_representations_test, global_representations_test= model.predict(X_test, batch_size = batch_size)

#simple classifier
model_D=tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = local_representations[0].shape),
    tf.keras.layers.Dense(1, activation = 'sigmoid')])



#load weights
model_D.load_weights('/content/drive/MyDrive/weights/model_D_weights')



/content/drive/MyDrive/protein-bert/proteinbert
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/miladansari/protein_bert.git to /tmp/pip-req-build-0h4jyvrh
  Running command git clone --filter=blob:none --quiet https://github.com/miladansari/protein_bert.git /tmp/pip-req-build-0h4jyvrh
  Resolved https://github.com/miladansari/protein_bert.git to commit 98738b2c0def5b16e750c758905b48b762523671
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
fatal: destination path 'shared_utils' already exists and is not an empty directory.
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


16/16 [==============================] - 4s 258ms/step


In [11]:
model_D.load_weights('/content/drive/MyDrive/weights/model_D_weights')